In [1]:
%load_ext autoreload
%autoreload 2

In [54]:
from unittest.mock import create_autospec
from typing import Protocol, Any

In [136]:
class IWeb(Protocol):
    def get(self, url: str) -> str:
        ...
    def request(self, url: str, body: dict[str, Any], verbose=False):
        ...

# Autospec

## Creating a mock based on a protocol

In [56]:
m = create_autospec(IWeb)

m.get('url')

<MagicMock name='mock.get()' id='2054874427504'>

## Calling function with invalid signature

In [57]:
try:
    m.get()
except Exception as e:
    print(e)

missing a required argument: 'url'


In [58]:
try:
    m.get(1,2)
except Exception as e:
    print(e)

too many positional arguments


## Accessing non-existing attributes

In [59]:
try:
    m.func
except Exception as e:
    print(e)

Mock object has no attribute 'func'


# Conditional return values

Global return values are done with:

In [60]:
m.get.return_value = 'response'

In [61]:
m.get('url')

'response'

But what if we only want a return type for a specific string?

In [62]:
def handle_return(*args, **kwargs):
    url = args[0]
    if url=='url':
        return 'handled response'
    return None

m.get.side_effect = handle_return

In [63]:
m.get(1)

In [64]:
m.get('url')

'handled response'

In [88]:
class ArgumentValidator:
    def __init__(self, validation_func, arg_name=None):
        self.validation_func = validation_func
        self.arg_name = arg_name
        
    @property
    def is_required(self):
        return self.arg_name is None
        
    def is_valid(self, arg):
        return self.validation_func(arg)

In [95]:
valid_dict = {'method': 1}
invalid_dict = {'method1': 1}

In [96]:
body_validator = ArgumentValidator(lambda d: 'method' in d.keys())
body_validator.is_valid(valid_dict), body_validator.is_valid(invalid_dict)

(True, False)

In [97]:
url_validator = ArgumentValidator(lambda s: s=='url')
url_validator.is_valid(1), url_validator.is_valid('url')

(False, True)

In [104]:
verbose_validator = ArgumentValidator(lambda b: b==True, arg_name='verbose')
verbose_validator.is_valid(True), verbose_validator.is_valid(False)

(True, False)

In [121]:
class SignatureValidator:
    def __init__(self, argument_validators):
        self.argument_validators = argument_validators
        self.required_validators = list(filter(lambda v: v.is_required, self.argument_validators))
        self.optional_validators = {val.arg_name: val for val in filter(lambda v: not v.is_required, self.argument_validators)}
        
    def is_valid(self, *args, **kwargs):
        if len(args)+len(kwargs) > len(self.argument_validators):
            return False
        
        if len(args) != len(self.required_validators):
            return False
        
        if len(kwargs) > len(self.optional_validators):
            return False
        
        positional_valid = all(arg_validator.is_valid(arg) for arg_validator,arg in zip(self.argument_validators, args))
        optional_valid = True
        
        for (arg_name, value) in kwargs.items():
            if arg_name not in self.optional_validators.keys():
                return False # actually raise error
            
            validator = self.optional_validators[arg_name]
            optional_valid = optional_valid and validator.is_valid(value)
        
        
        return positional_valid and optional_valid

In [122]:
s = SignatureValidator([url_validator, body_validator, verbose_validator])
s.is_valid('url1', invalid_dict), s.is_valid('url', valid_dict)

(False, True)

Invalid keyword arguments:

In [123]:
s.is_valid('url', valid_dict, verboser=False)

False

Non-matching keyword-arguments:

In [124]:
s.is_valid('url', valid_dict, verbose=False)

False

Matching signature

In [125]:
s.is_valid('url', valid_dict, verbose=True)

True

## Function introspection

In [137]:
IWeb.request.__kwdefaults__

In [138]:
IWeb.request.__defaults__

(False,)

In [142]:
def f(a=1):
    pass

def g(*, a=1):
    pass

In [143]:
f.__kwdefaults__, g.__kwdefaults__

(None, {'a': 1})

In [146]:
from inspect import signature

sig = signature(IWeb.request)
sig

<Signature (self, url: str, body: dict[str, typing.Any], verbose=False)>

In [147]:
sig.parameters

mappingproxy({'self': <Parameter "self">,
              'url': <Parameter "url: str">,
              'body': <Parameter "body: dict[str, typing.Any]">,
              'verbose': <Parameter "verbose=False">})

In [155]:
p = sig.parameters['url']
p, p.default

(<Parameter "url: str">, inspect._empty)

In [151]:
p = sig.parameters['verbose']

In [153]:
p.default

False